In [1]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import itertools

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression, PassiveAggressiveClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from pickle import dump, load

FILE_LOCATION = '../../Datasets/Combined_Dataset/combined_kaggle_corpus-20000.csv'

In [2]:
# csv_iter = pd.read_csv(FILE_LOCATION, engine='python', iterator=True, chunksize=100, dtype={'label': str, 'content': str})

# for chunk in csv_iter:
#     df = chunk
#     break

df = pd.read_csv(FILE_LOCATION, dtype={'label': str, 'content': str})
df.shape

(23973, 6)

# Data preprocessing

In [3]:
# ## Applying TFidf Vectorizer
# # tfidf_v=TfidfVectorizer(max_features=5000,ngram_range=(1,3))
# # X=tfidf_v.fit_transform(corpus).toarray()
# df ['domain'] = pd.to_numeric(df['domain'], errors='coerce')
# df ['title'] = pd.to_numeric(df['title'], errors='coerce')
# df ['content'] = pd.to_numeric(df['content'], errors='coerce')
# df ['url'] = pd.to_numeric(df['url'], errors='coerce')
# # df = df.dropna()
# df = df.replace(np.nan, 0, regex=True)
# # df.reset_index(inplace=True)

df = df.dropna()
df.reset_index(drop=True, inplace=True)
df = df.replace(to_replace='None', value=np.nan).dropna()
df['content'] = df['content'].astype('string')
df.dtypes

Unnamed: 0     int64
url           object
domain        object
title         object
content       string
label         object
dtype: object

In [4]:
df.head()

,Unnamed: 0,url,domain,title,content,label
0,0,https://blackagendareport.com/un-backed-police...,blackagendareport.com,UN polic massacr haitian with impun,UN polic massacr haitian with impun â polic fi...,fake
1,1,https://blackagendareport.com/articlelist/inte...,blackagendareport.com,black agenda report,It come surpris â US support war libya sell bl...,fake
2,2,https://blackagendareport.com/articlelist/inte...,blackagendareport.com,black agenda report,â polic final left campu around nine civilian ...,fake
3,3,https://blackagendareport.com/zambia-must-clar...,blackagendareport.com,zambia must clarifi whether It will host israe...,zambia must clarifi whether It will host israe...,fake
4,4,http://awm.com/woman-waves-hand-in-front-of-li...,awm.com,woman wave hand In front Of lizardâ tank captu...,most peopleâ pet member famili case dog cat lo...,fake


In [5]:
X = df['content']
X.head()

0    UN polic massacr haitian with impun â polic fi...
1    It come surpris â US support war libya sell bl...
2    â polic final left campu around nine civilian ...
3    zambia must clarifi whether It will host israe...
4    most peopleâ pet member famili case dog cat lo...
Name: content, dtype: string

In [6]:
Y = df['label']
Y.head()

0    fake
1    fake
2    fake
3    fake
4    fake
Name: label, dtype: object

In [7]:
tfidf_v=TfidfVectorizer(max_features=5000,ngram_range=(1,4))
X = tfidf_v.fit_transform(X).toarray()

In [8]:
## Divide the dataset into Train and Test
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

# VotingClassifier

In [12]:
svm = SGDClassifier(loss='hinge', alpha=1e-3, penalty='l2',random_state=42)
svc = LinearSVC()
lr = LogisticRegression()
clf = PassiveAggressiveClassifier(n_iter_no_change=50)
dt = DecisionTreeClassifier()

In [13]:
evc = VotingClassifier(estimators = [('svm',svm),('svc',svc),('lr',lr),('clf',clf),('dt',dt)], voting='hard')
evc.fit(X_train, Y_train)

VotingClassifier(estimators=[('svm',
                              SGDClassifier(alpha=0.001, random_state=42)),
                             ('svc', LinearSVC()), ('lr', LogisticRegression()),
                             ('clf',
                              PassiveAggressiveClassifier(n_iter_no_change=50)),
                             ('dt', DecisionTreeClassifier())])

In [14]:
print(evc.score(X_test, Y_test))
print(evc.score(X_train, Y_train))

0.9616080122409236
0.9974365088827948


In [15]:
dump(evc, open('votingclassifier_small.sav', 'wb'))